In [1]:
import os
import re
import tifffile as tf
from tqdm import tqdm
from IMC_Denoise.IMC_Denoise_main.DeepSNiF import DeepSNiF
import matplotlib.pyplot as plt

MARKER_CODE = ['176Yb-clPARP_Yb176.tiff', '148Nd-CD20_Nd148.tiff', '209Bi-CD15_Bi209.tiff', '175Lu-CD14_Lu175.tiff', '155Gd-GD2_Gd155.tiff', '191Ir-DNA1_Ir191.tiff', '152Sm-CD3_Sm152.tiff', '136Ba-crtl3_Ba136.tiff', '172Yb-IDO_Yb172.tiff', '154Sm-CD8a_Sm154.tiff', '131Xe-crtl1_Xe131.tiff', '163Dy-FOXP3_Dy163.tiff', '139La-Fibronectin_La139.tiff', '193Ir-DNA1_Ir193.tiff', '2IF_GD2-IF2.tiff', '115In-CD44_In115.tiff', '158Gd-CD10_Gd158.tiff', '145Nd-CXCR2_Nd145.tiff', '170Er-CD56_Er170.tiff', '143Nd-HLA-DR_Nd143.tiff', '89Y-MPO_Y89.tiff', '144Nd-PRPH_Nd144.tiff', '173Yb-GZMB_Yb173.tiff', '169Tm-Ki-67_Tm169.tiff', '168Er-GATA3_Er168.tiff', '134Xe-crtl2_Xe134.tiff', '162Dy-SOX10_Dy162.tiff', '196Pt-Vimentin_Pt196.tiff', '167Er-H4K12Ac_Er167.tiff', '147Sm-HLA-ABC_Sm147.tiff', '164Dy-CHGA_Dy164.tiff', '1IF_DAPI_IF1.tiff', '160Gd-CD274_Gd160.tiff', '166Er-PNMT_Er166.tiff', '161Dy-S100B_Dy161.tiff', '150Nd-CD11c_Nd150.tiff', '156Gd-CD34_Gd156.tiff', '146Nd-H3K9Ac_Nd146.tiff', '165Ho-CD279_Ho165.tiff', '174Yb-ELAVL4_Yb174.tiff', '149Sm-LUM_Sm149.tiff', '151Eu-CD24_Eu151.tiff', '153Eu-CD45_Eu153.tiff', '3IF_CD56-IF3.tiff', '171Yb-CD4_Yb171.tiff', '142Nd-CD11b_Nd142.tiff', '159Tb-CXCR4_Tb159.tiff']

Using TensorFlow backend.


In [2]:
base = "/data_isilon_main/isilon_images/10_MetaSystems/MetaSystemsData/Multimodal_Imaging_Daria/_Data_Analysis/_tmp_daria/Image_analysis/20230105_IMC-Denoise/"
trained_net_folder = os.path.join(base, "results")
image_folder = os.path.join(base, "training")

trained_markers = os.listdir(trained_net_folder)

rex = "[-].*[_]"

file_dict = {}
for marker in trained_markers:
    file = [x for x in MARKER_CODE if marker in x][0]
    protein = re.findall(rex, file)[0][1:-1]
    weights = os.path.join(trained_net_folder, marker, "weights.hdf5")
    file_dict[protein] = {"marker": marker, "file": file, "weights": weights} 

In [3]:
for k, v in file_dict.items():
    images = []
    for (dir_path, dir_names, file_names) in os.walk(image_folder):
        for file in file_names:
            if v["file"] in file:
                    images.append(os.path.join(dir_path, file))
    file_dict[k]["images"] = images

In [4]:
n_neighbours = 4 
n_iter = 3 
window_size = 3 

for k, v in file_dict.items():
    
    deepsnif = DeepSNiF(is_load_weights=True,
                        weights_name=v["weights"])
    
    for image in tqdm(v["images"]):
        
        raw_img = tf.imread(image)
        imc_denoise_img = deepsnif.perform_IMC_Denoise(raw_img, n_neighbours = n_neighbours, n_iter = n_iter, window_size = window_size)
        
        sample = image.split("/")[-2]
        
        save_dir = os.path.join(base, "evaluation", sample)
        
        if not os.path.isdir(save_dir):
            os.mkdir(save_dir)
            
        tf.imwrite(os.path.join(save_dir, f"{k}_denoised.tiff"), imc_denoise_img)
        tf.imwrite(os.path.join(save_dir, f"{k}_raw.tiff"), raw_img)
        
        # fig, ax = plt.subplots(1, 2, figsize=(10, 5))
        # ax[0].imshow(raw_img)
        # ax[1].imshow(imc_denoise_img)
        # plt.show()


Pre-trained model /data_isilon_main/isilon_images/10_MetaSystems/MetaSystemsData/Multimodal_Imaging_Daria/_Data_Analysis/_tmp_daria/Image_analysis/20230105_IMC-Denoise/results/142Nd/weights.hdf5 loaded successfully.
Pre-calculated range value file /data_isilon_main/isilon_images/10_MetaSystems/MetaSystemsData/Multimodal_Imaging_Daria/_Data_Analysis/_tmp_daria/Image_analysis/20230105_IMC-Denoise/results/142Nd/weights_range_val.npz loaded successfully.


100%|██████████| 150/150 [04:33<00:00,  1.83s/it]


Pre-trained model /data_isilon_main/isilon_images/10_MetaSystems/MetaSystemsData/Multimodal_Imaging_Daria/_Data_Analysis/_tmp_daria/Image_analysis/20230105_IMC-Denoise/results/169Tm/weights.hdf5 loaded successfully.
Pre-calculated range value file /data_isilon_main/isilon_images/10_MetaSystems/MetaSystemsData/Multimodal_Imaging_Daria/_Data_Analysis/_tmp_daria/Image_analysis/20230105_IMC-Denoise/results/169Tm/weights_range_val.npz loaded successfully.


100%|██████████| 150/150 [02:06<00:00,  1.19it/s]


Pre-trained model /data_isilon_main/isilon_images/10_MetaSystems/MetaSystemsData/Multimodal_Imaging_Daria/_Data_Analysis/_tmp_daria/Image_analysis/20230105_IMC-Denoise/results/151Eu/weights.hdf5 loaded successfully.
Pre-calculated range value file /data_isilon_main/isilon_images/10_MetaSystems/MetaSystemsData/Multimodal_Imaging_Daria/_Data_Analysis/_tmp_daria/Image_analysis/20230105_IMC-Denoise/results/151Eu/weights_range_val.npz loaded successfully.


...

100%|██████████| 150/150 [13:04<00:00,  5.23s/it]


Pre-trained model /data_isilon_main/isilon_images/10_MetaSystems/MetaSystemsData/Multimodal_Imaging_Daria/_Data_Analysis/_tmp_daria/Image_analysis/20230105_IMC-Denoise/results/139La/weights.hdf5 loaded successfully.
Pre-calculated range value file /data_isilon_main/isilon_images/10_MetaSystems/MetaSystemsData/Multimodal_Imaging_Daria/_Data_Analysis/_tmp_daria/Image_analysis/20230105_IMC-Denoise/results/139La/weights_range_val.npz loaded successfully.


100%|██████████| 150/150 [02:15<00:00,  1.11it/s]


Pre-trained model /data_isilon_main/isilon_images/10_MetaSystems/MetaSystemsData/Multimodal_Imaging_Daria/_Data_Analysis/_tmp_daria/Image_analysis/20230105_IMC-Denoise/results/209Bi/weights.hdf5 loaded successfully.
Pre-calculated range value file /data_isilon_main/isilon_images/10_MetaSystems/MetaSystemsData/Multimodal_Imaging_Daria/_Data_Analysis/_tmp_daria/Image_analysis/20230105_IMC-Denoise/results/209Bi/weights_range_val.npz loaded successfully.


100%|██████████| 150/150 [29:01<00:00, 11.61s/it]
